# Inteligencia Artificial

## Algoritmos Genéticos

### Integrantes

- Aldana S. Cardoso Rastrelli - 98.408
- Nicolás Continanza - 97.576


## Breve Marco Histórico

Dentro del campo de la Inteligencia Artificial y la resolución automática de problemas, en la década del '60 John Holland y su equipo de la Universidad de Michigan presentan los algoritmos genéticos, inspirados en la evolución biológica y la selección natural para problemas computacionales y de investigación operativa. El objetivo original de Holland no era el diseño de algoritmos para la resolución de problemas específicos, sino el estudio formal del fenómeno de adaptación como sucede en la naturaleza y el desarrollo de técnicas y métodos para importar mecanismos de adaptación natural a sistemas de computadoras.

En su libro _Adaptation in Natural and Artificial Systems (1975)_, Holland presenta un algoritmo genético como un método para pasar de una población de "cromosomas" (un conjunto de bits) a una nueva población evolucionada, mediante un proceso de selección natural junto con un conjunto de operadores inspirados en la genética (entrecruzamiento, mutación e inversión). Cada cromosoma consiste de "genes" (bits), cada uno siendo una instancia de un "alelo" (0 ó 1). El operador de selección elige aquellos cromosomas de la población que podrán reproducirse, y en promedio los cromosomas más aptos producen más descendencia que los menos aptos. El operador de crossover (entrecruzamiento, también llamado recombinación en buena parte de la literatura) intercambia subpartes de dos cromosomas, imitando la recombinación cromosómica entre dos haploides (células con un solo juego de cromosomas). El operador de mutación cambia aleatoriamente el valor de los alelos de algunas posiciones en el cromosoma. Y el operador de inversión invierte el orden de una sección contigua del cromosoma, cambiando el orden de los genes.

Con el pasar de los años hubo una amplia interacción entre investigadores que se dedicaron a estudiar variados métodos evolutivos computacionales, corriendo cada vez más los límites entre algoritmos genéticos, estrategias evolutivas y programación evolutiva, al punto de que al día de hoy el término _algoritmo genético_ se utiliza para describir cosas bastante lejanas a la concepción original de Holland, incorporando técnicas de machine learning para encontrar buenas (y a veces incluso óptimas) soluciones a problemas combinatorios con una inmensa cantidad de soluciones posibles.


## Algunas definiciones básicas

Comencemos con unas pocas y breves definiciones, que nos servirán para introducir un primer ejemplo utilizando el vocabulario específico del tema.

#### Población

Dado un problema combinatorio, en el contexto de Algoritmos Genéticos llamaremos _población_ al conjunto de soluciones posibles para ese problema. En el marco de este Trabajo Práctico, ese conjunto siempre será discreto y finito.

#### Cromosoma

Un _Cromosoma_, _Individuo_ o _Genotipo_ (los tres términos se utilizan como sinónimos en la literatura) es un elemento particular de la población. Se lo suele representar como una combinación de bits, a la que se denomina _Genoma_.

#### Gen

Si consideramos un cromosoma como una combinación de bits, llamaremos _Gen_ a cada posición de esa combinación. Sus posibles valores, 0 ó 1, son conocidos como _Alelos_.


## Un primer ejemplo

Los algoritmos genéticos utilizan la exploración aleatoria del espacio del problema, combinando procesos evolutivos como la mutación y el crossover para mejorar las conjeturas. Pero también, al no tener experiencia en el dominio del problema, intentan cosas que un humano nunca intentaría. De esta manera, una persona usando un algoritmo genético puede aprender más acerca del espacio del problema y potenciales soluciones, pudiendo así realizar mejoras al algoritmo en un círculo virtuoso.

Tomemos como ejemplo inicial el ejercicio de adivinar una contraseña. Para simplificar el problema y poner el foco en lo que nos interesa, conocemos de antemano la longitud de la contraseña. Para ayudar a introducir algunos conceptos iniciales, después de cada intento tendremos la posibilidad de saber cuántas letras estaban en la posición correcta. Por ejemplo, si la contraseña es "Hello World!" y se intenta con la cadena "World!Hello?" se obtendrá el número 2, ya que solo la cuarta letra de cada palabra está en la posición correcta.

Pseudocódigo:

```
_letters = [a..zA..Z !]
target = "Hello World!"
guess = get 12 random letters from _letters
while guess != target:
  index = get random value from [0..length of target]
  guess[index] = get 1 random value from _letters
```

### Genes

Para comenzar, los algoritmos genéticos necesitan un set de genes para construir las posibles soluciones. Para este ejemplo inicial, el set de genes serán las letras del abecedario. También es necesario un target (la contraseña a adivinar):


In [8]:
geneSet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
target = "Hello World!"


A continuación, el algoritmo deberá tener una manera de generar una cadena aleatoria a partir del set de genes, que será el punto de partida del proceso evolutivo. Por eso se lo llama _Generación 0_.


In [9]:
import random


def generate_parent(length):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)


La función `random.sample()` toma una cantidad `sampleSize` de valores de la entrada sin reposición. Por lo tanto no habrá valores repetidos en el parent generado, salvo que `geneSet` contenga repetidos o `length` sea mayor que `len(geneSet)`. Esta implementación puede generar una larga cadena de caracteres a partir de un pequeño conjunto de genes, y usa tantos genes únicos como sea posible.


### Función de Aptitud (Fitness Function)

El valor de fitness que provee el algoritmo genético es el único feedback que el sistema obtiene para guiarse hacia una solución. En este ejemplo, el valor de fitness es la cantidad total de letras en un intento que coinciden con la letra del target en la misma posición.


In [10]:
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target, guess)
               if expected == actual)


### Mutación

A continuación, el sistema requiere de una forma de producir un nuevo intento mutando el actual. Es decir, "avanzar" una generación. La siguiente función convierte la cadena `parent` en un array, y luego reemplaza una letra del array con una elegida aleatoriamente de `geneSet`, recombinando el resultado nuevamente en una cadena de caracteres.


In [11]:
def mutate(parent):
    index = random.randrange(0, len(parent))
    childGenes = list(parent)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate \
        if newGene == childGenes[index] \
        else newGene
    return ''.join(childGenes)


Se usa un reemplazo alternativo en caso de que `newGene` (seleccionado aleatoriamente) sea igual al que se supone que reemplazará, para prevenir un número significativo de intentos irrelevantes.


### Mostrando resultados

Queremos monitorear el paso a paso del algoritmo para tener una noción de su avance y sacar conclusiones. Una representación visual de la secuencia genética suele ser crítico para identificar qué funciona y qué no, para que el algoritmo pueda ser mejorado.

En nuestro caso, mostraremos también el valor de fitness y cuánto tiempo transcurrió.


In [12]:
import datetime


def display(guess, startTime):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(guess)
    print("{0}\t{1}\t{2}".format(guess, fitness, str(timeDiff)))


Pasemos a poner en práctica nuestro algoritmo y analizar los resultados obtenidos.
En esencia, el algoritmo consistirá de los siguientes pasos:

1. Generar un intento (Generación 0).
2. Obtener el valor `fitness` para ese intento.
3. Comparar el valor `fitness` obtenido con el del intento anterior.
4. Conservar el intento con mejor `fitness` (Selección Natural).

Esto se repite en un ciclo hasta que ocurra una condición de corte, que en nuestro caso es haber encontrado la contraseña correcta (`fitness = 12`).


In [14]:
random.seed()
startTime = datetime.datetime.now()
bestParent = generate_parent(len(target))
bestFitness = get_fitness(bestParent)
display(bestParent, startTime)

while True:
    child = mutate(bestParent)
    childFitness = get_fitness(child)
    if bestFitness >= childFitness:
        continue
    display(child, startTime)
    if childFitness >= len(bestParent):
        break
    bestFitness = childFitness
    bestParent = child


gDnKFOaUXQeS	0	0:00:00.000143
gDnKFOaUXleS	1	0:00:00.000326
gDnKFOaoXleS	2	0:00:00.000361
gDnKoOaoXleS	3	0:00:00.000728
gDnKoOWoXleS	4	0:00:00.001883
gDnKo WoXleS	5	0:00:00.002605
gDnlo WoXleS	6	0:00:00.002962
gDllo WoXleS	7	0:00:00.003770
gello WoXleS	8	0:00:00.004342
gello WoXldS	9	0:00:00.004413
gello WoXld!	10	0:00:00.004744
Hello WoXld!	11	0:00:00.005743
Hello World!	12	0:00:00.013152


Vemos que obtener una contraseña sencilla con este elemental algoritmo genético tomó poco más de 1 centésima de segundo.


## Solución al Problema de la Mochila usando un algoritmo genético

A continuación, un ejemplo más difícil que nos permitirá ver con más claridad la potencia de esta técnica: el Problema de la Mochila.

Partimos del problema clásico: tenemos un conjunto de ítems que queremos guardar en una mochila. Para cada ítem, conocemos su peso y le agregamos un valor que representa la importancia de ese ítem. Por otro lado tenemos una mochila, que tiene un límite de peso que puede soportar. Queremos construir un algoritmo que nos permita encontrar la combinación de ítems a meter en la mochila, de forma tal que se maximice el valor total sin exceder el límite de peso.

Comencemos por la representación genética de una solución: aquí nuestra _población_ son todas las posibles combinaciones de los ítems disponibles, donde cada una de ellas es un _cromosoma_.

Si bien Python es un lenguaje de tipado dinámico que no obliga a explicitar los tipos, lo haremos con la biblioteca [`typing`](https://docs.python.org/3/library/typing.html) para favorecer la claridad y comprensión del código al introducir el vocabulario específico del dominio de los algoritmos genéticos.


In [3]:
from typing import Callable, List, NamedTuple, Tuple

Genome = List[int]
Population = List[Genome]
FitnessFunction = Callable[[Genome], int]
PopulateFunction = Callable[[], Population]
SelectionFunction = Callable[[Population,
                              FitnessFunction], Tuple[Genome, Genome]]
CrossoverFunction = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunction = Callable[[Genome], Genome]
Thing = NamedTuple(
    'Thing', [('name', str), ('value', float), ('weight', float)])


things = [
    Thing('Laptop', 500, 2200),
    Thing('Headphones', 150, 160),
    Thing('Coffee Mug', 60, 350),
    Thing('Notepad', 40, 333),
    Thing('Water Bottle', 30, 192),
]

more_things = [
    Thing('Mints', 5, 25),
    Thing('Socks', 10, 38),
    Thing('Tissues', 15, 80),
    Thing('Phone', 500, 200),
    Thing('Baseball Cap', 100, 70)
] + things


Como en todos los problemas combinatorios, una de las dificultades para encontrar soluciones óptimas reside en la magnitud del conjunto de soluciones posibles. Este problema no es la excepción, ya que a medida que crece la cantidad de cosas que nos interesaría llevar en la mochila, crece exponencialmente la cantidad de combinaciones posible.

Es por esto que para la generación 0 usaremos una población de tamaño fijo, construida a partir de genomas generados aleatoriamente.


In [4]:
from random import choices


def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)


def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]


Para la función de fitness nos aseguraremos que el genoma tenga tantos genes como elementos queremos seleccionar para meter en la mochila, y luego calcularemos el valor total de los elementos que ese genoma inserta en la mochila como la suma de los valores de cada ítem individual. Si la combinación del genoma dado excede el límite de peso, el valor total es 0 para descartar fácilmente ese genoma.


In [5]:
def fitness(genome: Genome, things: [Thing], weight_limit: float) -> float:
    if len(genome) != len(things):
        raise ValueError("Genome length must be equal to number of things")
    weight = 0
    value = 0

    for i, thing in enumerate(things):
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value

            if weight > weight_limit:
                return 0

    return value


A continuación construiremos una función de selección, que nos servirá para seleccionar las soluciones que conformarán la próxima generación. Usaremos nuevamente la función choices(), pero esta vez la distribución de probabilidad no será uniforme sino que estará influida por la función de fitness, para favorecer la selección de aquellos genomas más "aptos".


In [6]:
def selection_pair(population: Population, fitness_function: FitnessFunction) -> Population:
    return choices(population=population, weights=[fitness_function(genome) for genome in population], k=2)


Ahora necesitamos una función de crossover, que nos permita combinar dos genomas para producir uno nuevo. Para ello, elegiremos un punto de corte aleatorio y luego intercambiaremos los genes de cada genoma a partir de ese punto.


In [7]:
from random import randint


def single_point_cross_over(parent1: Genome, parent2: Genome) -> Tuple[Genome, Genome]:
    if len(parent1) != len(parent2):
        raise ValueError("Both genomes must be of equal length")
    cross_over_point = randint(1, len(parent1) - 1)
    return [parent1[:cross_over_point] + parent2[cross_over_point:], parent2[:cross_over_point] + parent1[cross_over_point:]]


Finalmente, una función de mutación que cambia un gen al azar en un genoma, con una cierta probabilidad.


In [16]:
from random import randrange, random


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


Ahora pondremos todas estas funciones en uso, y veremos cómo se comporta el algoritmo genético para resolver el problema de la mochila simulando la evolución de una población de genomas. Vamos a aprovechar las funciones de alto orden en Python y generalizar para que la simulación de la evolución pueda realizarse con cualquier función que se reciba como parámetro. Tendremos además un límite de generaciones, para forzar un corte si el algoritmo no logra encontrar una solución óptima antes.


In [17]:
def run_evolution(
        populate_function: PopulateFunction,
        fitness_function: FitnessFunction,
        fitness_limit: int,
        selection_function: SelectionFunction = selection_pair,
        crossover_function: CrossoverFunction = single_point_cross_over,
        mutation_function: MutationFunction = mutation,
        generation_limit: int = 100,
) -> Tuple[Population, int]:
    population = populate_function()
    for i in range(generation_limit):
        population = sorted(
            population, key=lambda genome: fitness_function(genome), reverse=True)
        if fitness_function(population[0]) >= fitness_limit:
            return population, i
        next_generation = population[0:2]

        for j in range(int(len(population) - 2) - 1):
            parents = selection_function(population, fitness_function)
            children = crossover_function(parents[0], parents[1])
            children = [mutation_function(child) for child in children]
            next_generation.extend(children)
        population = next_generation

    population = sorted(
        population, key=lambda genome: fitness_function(genome), reverse=True
    )

    return population, i


In [18]:
from functools import partial
import time


start = time.time()
population, generations = run_evolution(
    populate_function=partial(generate_population, size=10, genome_length=len(things)),
    fitness_function=partial(fitness, things=things, weight_limit=3000),
    fitness_limit=740,
    generation_limit=100
)
end = time.time()

print(f"Number of generations: {generations}")
print(f"Time: {end - start}s")
print(f"Best solution: {population[0]}")

Number of generations: 1
Time: 0.0001480579376220703s
Best solution: [1, 1, 1, 0, 1]


In [20]:
start = time.time()
population, generations = run_evolution(
    populate_function=partial(
        generate_population, size=10, genome_length=len(more_things)),
    fitness_function=partial(fitness, things=more_things, weight_limit=3000),
    fitness_limit=1310,
    generation_limit=100
)
end = time.time()

print(f"Number of generations: {generations}")
print(f"Time: {end - start}s")
print(f"Best solution: {population}")


Number of generations: 6
Time: 0.04417896270751953s
Best solution: [[1, 1, 1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 1], [0, 1, 1, 1, 1, 1, 1, 0, 0, 1], [0, 1, 1, 1, 1, 1, 1, 0, 0, 1], [0, 1, 1, 1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [0, 0, 1, 1, 1, 1, 1, 0, 0, 1], [0, 0, 1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 0, 1, 1, 1, 1, 0, 0, 1], [0, 0, 1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 0, 1, 1, 1, 1, 0, 0, 1], [0, 1, 0, 1, 1, 1, 1, 0, 0, 1], [0, 1, 0, 1, 1, 1, 1, 0, 0, 1], [0, 1, 0, 1, 1, 1, 1, 0, 0, 1], [0, 0, 0, 1, 1, 1, 1, 0, 0, 1], [0, 0, 1, 1, 1, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 1, 1, 0, 0, 0], [0, 1, 0, 1, 0, 1, 1, 1, 0, 0], [0, 1, 0, 1, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 0, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 1, 1, 0, 0, 1], [0, 0, 0, 1, 0, 1, 1, 1, 0, 0], [1, 

## Bibliografía

- R. Poli, W. B. Langdon, and N. F. McPhee. _A field guide to genetic programming_. 2008. (With contributions by J. R. Koza). Publicado via http://lulu.com y disponible gratuitamente en http://www.gp-field-guide.org.uk
